In [1]:
!pip install opendatasets datasets accelerate --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [2]:
import opendatasets as od
od.download("https://www.kaggle.com/datasets/gondimalladeepesh/nvidia-documentation-question-and-answer-pairs")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: dholesneha
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/gondimalladeepesh/nvidia-documentation-question-and-answer-pairs


100%|██████████| 400k/400k [00:00<00:00, 1.04MB/s]

In [2]:
import pandas as pd
from datasets import Dataset
import datasets
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
import torch
import time
import re

device = 'cuda' if torch.cuda.is_available() else 'cpu' # detect the GPU if any, if not use CPU, change cuda to mps if you have a mac

In [3]:
import transformers
print(transformers.__version__)


4.42.4


In [5]:
device

'cuda'

In [4]:
data = pd.read_csv("/content/nvidia-documentation-question-and-answer-pairs/NvidiaDocumentationQandApairs.csv")[["question", "answer"]]
print(data.shape)
data.head()

(7108, 2)


,question,answer
0,What is Hybridizer?,Hybridizer is a compiler from Altimesh that en...
1,How does Hybridizer generate optimized code?,Hybridizer uses decorated symbols to express p...
2,What are some parallelization patterns mention...,The text mentions using parallelization patter...
3,How can you benefit from accelerators without ...,You can benefit from accelerators' compute hor...
4,What is an example of using Hybridizer?,An example in the text demonstrates using Para...


In [5]:
def clean_text(text):
  text = text.lower()
  text = re.sub('[^A-Za-z0-9]+', ' ', text)
  return text

data['question'] = data['question'].apply(clean_text)
data['answer'] = data['answer'].apply(clean_text)
data.head()

,question,answer
0,what is hybridizer,hybridizer is a compiler from altimesh that en...
1,how does hybridizer generate optimized code,hybridizer uses decorated symbols to express p...
2,what are some parallelization patterns mention...,the text mentions using parallelization patter...
3,how can you benefit from accelerators without ...,you can benefit from accelerators compute hors...
4,what is an example of using hybridizer,an example in the text demonstrates using para...


In [6]:
train=data.sample(frac=0.7,random_state=7) # Create training of 70% of the data
test=data.drop(train.index) # Create testing by removing the 70% of the train data which will result in 30%

val=test.sample(frac=0.5,random_state=7) # Create validation of 50% of the testing data
test=test.drop(val.index) # Create testing by removing the 50% of the validation data which will result in 50%

In [7]:
print("Training Shape: ", train.shape)
print("Validation Shape: ", val.shape)
print("Testing Shape: ", test.shape)

Training Shape:  (4976, 2)
Validation Shape:  (1066, 2)
Testing Shape:  (1066, 2)


In [8]:
def get_model_tokenizer(model_name = "google/flan-t5-base"):
  original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.float32)
  tokenizer = AutoTokenizer.from_pretrained(model_name)

  return original_model, tokenizer

In [9]:
model, tokenizer = get_model_tokenizer()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [11]:
def tokenize_function(example):
    start_prompt = '\n\n'
    end_prompt = '\n\nAnswer: '
    prompt = [start_prompt + dialogue + end_prompt for dialogue in example["question"]]
    example['input_ids'] = tokenizer(prompt, padding="max_length", truncation=True, return_tensors="pt", max_length=200).input_ids
    example['labels'] = tokenizer(example["answer"], padding="max_length", truncation=True, return_tensors="pt",max_length=200).input_ids

    return example

In [14]:
train_data = Dataset.from_pandas(train)
train_tokenized_datasets = train_data.map(tokenize_function, batched=True)
train_tokenized_datasets = train_tokenized_datasets.remove_columns(['question', 'answer','__index_level_0__'])


val_data = Dataset.from_pandas(val)
val_tokenized_datasets = val_data.map(tokenize_function, batched=True)
val_tokenized_datasets = val_tokenized_datasets.remove_columns(['question', 'answer','__index_level_0__'])


test_data = Dataset.from_pandas(test)
test_tokenized_datasets = test_data.map(tokenize_function, batched=True)
test_tokenized_datasets = test_tokenized_datasets.remove_columns(['question', 'answer','__index_level_0__'])

Map:   0%|          | 0/4976 [00:00<?, ? examples/s]

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

In [16]:
print(train_tokenized_datasets)

print(val_tokenized_datasets)

print(test_tokenized_datasets)

Dataset({
    features: ['input_ids', 'labels'],
    num_rows: 4976
})
Dataset({
    features: ['input_ids', 'labels'],
    num_rows: 1066
})
Dataset({
    features: ['input_ids', 'labels'],
    num_rows: 1066
})


In [17]:
EPOCHS = 1
LR = 1e-4
BATCH_SIZE = 2

training_path = "./training_path_nvidia_chatbot"
print(training_path)

training_args = TrainingArguments(
    output_dir = training_path,
    overwrite_output_dir = True,
    per_device_train_batch_size = BATCH_SIZE,
    per_device_eval_batch_size = BATCH_SIZE,
    learning_rate = LR,
    num_train_epochs = EPOCHS,
    evaluation_strategy = "epoch",
    save_total_limit = 2
    )

trainer= Trainer(
    model = model,
    args = training_args,
    train_dataset = train_tokenized_datasets,
    eval_dataset = val_tokenized_datasets,
)


./training_path_nvidia_chatbot


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [18]:

trainer.train()

model_path = "./nvidia-chatbot-final-model"

trainer.model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path) ##not required

Epoch,Training Loss,Validation Loss
1,0.537100,0.438890


('./nvidia-chatbot-final-model/tokenizer_config.json',
 './nvidia-chatbot-final-model/special_tokens_map.json',
 './nvidia-chatbot-final-model/spiece.model',
 './nvidia-chatbot-final-model/added_tokens.json',
 './nvidia-chatbot-final-model/tokenizer.json')

In [20]:
# rouge_metric = datasets.load_metric("rouge")
eval_results= trainer.evaluate(
    eval_dataset = test_tokenized_datasets,
                 )

In [21]:
print(eval_results)

{'eval_loss': 0.4373830258846283, 'eval_runtime': 47.2193, 'eval_samples_per_second': 22.576, 'eval_steps_per_second': 11.288, 'epoch': 1.0}


##Model Inference

In [ ]:
test_text = "what is cuda nsight?"
trained_model, tokenizer = get_model_tokenizer(model_path)

tokenized_test_text = tokenizer(test_text,
                              return_tensors='pt')
model_output = trained_model.generate(tokenized_test_text.input_ids,
                                      generation_config=GenerationConfig(max_new_tokens=200, num_beams=1),)[0]
final_output = tokenizer.decode(model_output, skip_special_tokens=True)
print(final_output)